<center>
    <h1> ILI-286 - Laboratorio #5 </h1>
    <h2> Sistemas de reacción-difusión </h2>
</center>


| Nombre | Rol | Email |
| :----- | :-- | :---- |
| Marco Rojas | 201073005-0 | marco.rojaso@alumnos.usm.cl |
| Hernán Vargas | 201073009-3 | hernan.vargas@alumnos.usm.cl |

## Tabla de contenidos
* [Introducción](#intro)
* [Desarrollo y analisís de resultados:](#desarrollo)
  1. [Simulando Sistemas Caóticos](#se1)
  2. [BVP de orden superior](#se2)
* [Conclusiones](#Conclusiones)
* [Referencias](#Referencias)
* [Anexos](#Anexos) $\leftarrow$ Comenzar ejecución aquí
  * [Métodos implementados](#an1)
  * [Métodos paralelos](#an2)
  * [Visualización de resultados](#an3)
  * [Implementación de diferencias finitas](#an4)
  * [Utilidades](#an5)

<div id='intro' />
## Introducción
#ToDo

<div id='desarrollo'/>
## Desarrollo y analisís de resultados

<div id='se1'/>
## 1.- Diferencias finitas en 2D

$$ \frac{\partial u}{\partial t} = D_u \, (\frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2}) + \lambda \, u - u^3 - \kappa - \sigma v $$
$$ \tau \, \frac{\partial v}{\partial t} = D_v \, (\frac{\partial^2 v}{\partial x^2} + \frac{\partial^2 v}{\partial y^2}) + u - v $$

### a) Propuesta de discretización "Forward"
Aproximaciones:
$$ \frac{\partial^2 u}{\partial x^2} = \frac{u(x+\Delta x, y, t)-2u(x,y,t)+u(x-\Delta x,y,t)}{\Delta x^2} $$

$$ \frac{\partial^2 u}{\partial y^2} = \frac{u(x, y+\Delta y, t)-2u(x,y,t)+u(x,y-\Delta y,t)}{\Delta y^2} $$

$$ \frac{\partial u}{\partial t} = \frac{u(x,y,t+\Delta t) - u(x,y,t)}{\Delta t} $$

Análogo para $v(x,y,t)$.

Las ecuaciones quedan como:

$$ \frac{u(x,y,t+\Delta t) - u(x,y,t)}{\Delta t} = D_u \, \left (\frac{u(x+\Delta x, y, t)-2u(x,y,t)+u(x-\Delta x,y,t)}{\Delta x^2} + \frac{u(x, y+\Delta y, t)-2u(x,y,t)+u(x,y-\Delta y,t)}{\Delta y^2} \right ) + \lambda \, u(x,y,t) - u(x,y,t)^3 - \kappa - \sigma v(x,y,t) $$

$$ \tau \, \frac{v(x,y,t+\Delta t) - v(x,y,t)}{\Delta t} = D_v \, \left ( \frac{v(x+\Delta x,y,t)-2v(x,y,t)+v(x-\Delta x,y,t)}{\Delta x^2} + \frac{v(x,y+\Delta y,t)-2v(x,y,t)+v(x,y-\Delta y,t)}{\Delta y^2} \right ) + u(x,y,t) - v(x,y,t) $$

Ahora si reemplazamos $w_{i,j,k} = u(x,y,t)$ y $ b_{i,j,k} = v(x,y,t) $, donde 
$$x= x_0+i \,\Delta x \Rightarrow i= \frac{x-x_0}{\Delta x}$$
$$y= y_0+j \,\Delta y \Rightarrow j= \frac{y-y_0}{\Delta y}$$
$$t= t_0+k \,\Delta t \Rightarrow k= \frac{t-t_0}{\Delta t}$$

Finalmente, obtenemos

$$ \frac{w_{i,j,k+1} - w_{i,j,k}}{\Delta t} = D_u  \, \left (\frac{w_{i+1, j, k}-2w_{i,j,k}+w_{i-1,j,k}}{\Delta x^2} + \frac{w_{i, j+1, k}-2w_{i,j,k}+w_{i,j-1,k}}{\Delta y^2} \right ) + \lambda \, w_{i,j,k} - w_{i,j,k}^3 - \kappa - \sigma b_{i,j,k} $$

$$ \tau \, \frac{b_{i,j,k+1} - b_{i,j,k}}{\Delta t} = D_v \, \left ( \frac{b_{i+1,j,k}-2b_{i,j,k}+b_{i-1,j,k}}{\Delta x^2} + \frac{b_{i,j+1,k}-2b_{i,j,k}+b_{i,j-1,k}}{\Delta y^2} \right ) + w_{i,j,k} - b_{i,j,k} $$

Despejando para el tiempo $k+1$


$$ w_{i,j,k+1} = \Delta t \left ( D_u  \, \left (\frac{w_{i+1, j, k}-2w_{i,j,k}+w_{i-1,j,k}}{\Delta x^2} + \frac{w_{i, j+1, k}-2w_{i,j,k}+w_{i,j-1,k}}{\Delta y^2} \right ) + \lambda \, w_{i,j,k} - w_{i,j,k}^3 - \kappa - \sigma b_{i,j,k} \right ) + w_{i,j,k} $$

$$ b_{i,j,k+1} = \frac{\Delta t}{\tau} \left ( D_v \, \left ( \frac{b_{i+1,j,k}-2b_{i,j,k}+b_{i-1,j,k}}{\Delta x^2} + \frac{b_{i,j+1,k}-2b_{i,j,k}+b_{i,j-1,k}}{\Delta y^2} \right ) + w_{i,j,k} - b_{i,j,k} \right ) + b_{i,j,k}$$


El error de aproximación es de: $O(\Delta t) + O(\Delta x^2) + O(\Delta y^2)$

### b) Implementación de esquema explícito

In [ ]:
def solve_forward(u0, v0, dx, dy, dt, Tf, Du, Dv, lamb, tau, sigma, kappa):
    Ny = u0.shape[0];
    Nx = u0.shape[1];
    Nt = (int)(np.ceil(Tf/dt));
    w = np.zeros([Nt, Nx, Ny])
    b = np.zeros([Nt, Nx, Ny])
    w[0,:,:] = u0;
    b[0,:,:] = v0;
    for k in range (Nt-1):
        u_xx = (np.roll(w[k,:,:], 1, axis=1) - 2 * w[k,:,:] + np.roll(w[k,:,:], -1, axis=1))/dx**2;
        u_yy = (np.roll(w[k,:,:], 1, axis=0) - 2 * w[k,:,:] + np.roll(w[k,:,:], -1, axis=0))/dy**2;
        u_laplaciano = u_xx + u_yy;
        
        v_xx = (np.roll(b[k, :, :], 1, axis=1) - 2 * b[k,:,:] + np.roll(b[k,:,:], -1, axis=1))/dx**2;
        v_yy = (np.roll(b[k, :, :], 1, axis=0) - 2 * b[k,:,:] + np.roll(b[k,:,:], -1, axis=0))/dy**2;
        v_laplaciano = v_xx + v_yy
        
        w[k+1,:,:] = dt * (Du * u_laplaciano + lamb * w[k,:,:] - np.power(w[k,:,:],3) - kappa - sigma * b[k,:,:]) + w[k,:,:];
        b[k+1,:,:] = dt/tau * (Dv * v_laplaciano + w[k,:,:] - b[k,:,:]) + b[k,:,:];
        
    return w;
u0 = np.matrix([[2,3,5],[1,2,3],[2,3,4]]);
v0 = np.matrix([[1,3,1],[2,2,1],[2,3,1]]);
Du = np.matrix([[4,0,0],[0,7,0],[0,0,9]]);
Dv = np.matrix([[2,0,0],[0,3,0],[0,0,2]]);
print(u0);
print(solve_forward(u0, v0, 1, 1, 1, 4, Du, Dv, 8, 5, 3, 3));

### c) Propuesta de discretización "Backward"
Aproximaciones:
$$ \frac{\partial^2 u}{\partial x^2} = \frac{u(x+\Delta x, y, t)-2u(x,y,t)+u(x-\Delta x,y,t)}{\Delta x^2} $$

$$ \frac{\partial^2 u}{\partial y^2} = \frac{u(x, y+\Delta y, t)-2u(x,y,t)+u(x,y-\Delta y,t)}{\Delta y^2} $$

$$ \frac{\partial u}{\partial t} = \frac{u(x,y,t) - u(x,y,t-\Delta t)}{\Delta t} $$

Análogo para $v(x,y,t)$.

Las ecuaciones quedan como:

$$ \frac{u(x,y,t) - u(x,y,t - \Delta t)}{\Delta t} = D_u \, \left (\frac{u(x+\Delta x, y, t)-2u(x,y,t)+u(x-\Delta x,y,t)}{\Delta x^2} + \frac{u(x, y+\Delta y, t)-2u(x,y,t)+u(x,y-\Delta y,t)}{\Delta y^2} \right ) + \lambda \, u(x,y,t) - u(x,y,t)^3 - \kappa - \sigma v(x,y,t) $$

$$ \tau \, \frac{v(x,y,t) - v(x,y,t-\Delta t)}{\Delta t} = D_v \, \left ( \frac{v(x+\Delta x,y,t)-2v(x,y,t)+v(x-\Delta x,y,t)}{\Delta x^2} + \frac{v(x,y+\Delta y,t)-2v(x,y,t)+v(x,y-\Delta y,t)}{\Delta y^2} \right ) + u(x,y,t) - v(x,y,t) $$

Ahora si reemplazamos $w_{i,j,k} = u(x,y,t)$ y $ b_{i,j,k} = v(x,y,t) $, donde 
$$x= x_0+i \,\Delta x \Rightarrow i= \frac{x-x_0}{\Delta x}$$
$$y= y_0+j \,\Delta y \Rightarrow j= \frac{y-y_0}{\Delta y}$$
$$t= t_0+k \,\Delta t \Rightarrow k= \frac{t-t_0}{\Delta t}$$

Finalmente, obtenemos

$$ \frac{w_{i,j,k} - w_{i,j,k-1}}{\Delta t} = D_u  \, \left (\frac{w_{i+1, j, k}-2w_{i,j,k}+w_{i-1,j,k}}{\Delta x^2} + \frac{w_{i, j+1, k}-2w_{i,j,k}+w_{i,j-1,k}}{\Delta y^2} \right ) + \lambda \, w_{i,j,k} - w_{i,j,k}^3 - \kappa - \sigma b_{i,j,k} $$

$$ \tau \, \frac{b_{i,j,k} - b_{i,j,k-1}}{\Delta t} = D_v \, \left ( \frac{b_{i+1,j,k}-2b_{i,j,k}+b_{i-1,j,k}}{\Delta x^2} + \frac{b_{i,j+1,k}-2b_{i,j,k}+b_{i,j-1,k}}{\Delta y^2} \right ) + w_{i,j,k} - b_{i,j,k} $$

Cambiando $k$ por $k+1$

$$ \frac{w_{i,j,k+1} - w_{i,j,k}}{\Delta t} = D_u  \, \left (\frac{w_{i+1, j, k+1}-2w_{i,j,k+1}+w_{i-1,j,k+1}}{\Delta x^2} + \frac{w_{i, j+1, k+1}-2w_{i,j,k+1}+w_{i,j-1,k+1}}{\Delta y^2} \right ) + \lambda \, w_{i,j,k+1} - w_{i,j,k+1}^3 - \kappa - \sigma b_{i,j,k+1} $$

$$ \tau \, \frac{b_{i,j,k+1} - b_{i,j,k}}{\Delta t} = D_v \, \left ( \frac{b_{i+1,j,k+1}-2b_{i,j,k+1}+b_{i-1,j,k+1}}{\Delta x^2} + \frac{b_{i,j+1,k+1}-2b_{i,j,k+1}+b_{i,j-1,k+1}}{\Delta y^2} \right ) + w_{i,j,k+1} - b_{i,j,k+1} $$

Despejando para el tiempo $k+1$


$$ w_{i,j,k+1} - \Delta t \left ( D_u  \, \left (\frac{w_{i+1, j, k+1}-2w_{i,j,k+1}+w_{i-1,j,k+1}}{\Delta x^2} + \frac{w_{i, j+1, k+1}-2w_{i,j,k+1}+w_{i,j-1,k+1}}{\Delta y^2} \right ) + \lambda \, w_{i,j,k+1} - w_{i,j,k+1}^3 - \kappa - \sigma b_{i,j,k+1} \right ) = w_{i,j,k} $$

$$ b_{i,j,k+1} - \frac{\Delta t}{\tau} \left ( D_v \, \left ( \frac{b_{i+1,j,k+1}-2b_{i,j,k+1}+b_{i-1,j,k+1}}{\Delta x^2} + \frac{b_{i,j+1,k+1}-2b_{i,j,k+1}+b_{i,j-1,k+1}}{\Delta y^2} \right ) + w_{i,j,k+1} - b_{i,j,k+1} \right )= b_{i,j,k}$$

Agrupando incógnitas en la primera ecuación

$$ w_{i,j,k+1} - \frac{\Delta t \, D_u \, w_{i+1, j, k+1}}{\Delta x^2} + 2\frac{\Delta t \, D_u \, w_{i,j,k+1}}{\Delta x^2} - \frac{\Delta t \, D_u \, w_{i-1,j,k+1}}{\Delta x^2} - \frac{\Delta t \, D_u \, w_{i, j+1, k+1}}{\Delta y^2}+2 \frac{\Delta t \, D_u \, w_{i,j,k+1}}{\Delta y^2} - \frac{\Delta t \, D_u \, w_{i,j-1,k+1}}{\Delta y^2} - \Delta t \, \lambda \, w_{i,j,k+1} + \Delta t \, w_{i,j,k+1}^3 + \Delta t \, \kappa + \Delta t \, \sigma b_{i,j,k+1}  = w_{i,j,k} $$


$$ (1+ 2\frac{\Delta t \, D_u}{\Delta x^2} + 2 \frac{\Delta t \, D_u}{\Delta y^2 } - \Delta t \, \lambda + \Delta t \, w_{i,j,k+1}^2) \, w_{i,j,k+1} - \frac{\Delta t \, D_u \, w_{i+1, j, k+1}}{\Delta x^2} - \frac{\Delta t \, D_u \, w_{i-1,j,k+1}}{\Delta x^2} - \frac{\Delta t \, D_u \, w_{i, j+1, k+1}}{\Delta y^2} - \frac{\Delta t \, D_u \, w_{i,j-1,k+1}}{\Delta y^2} + \Delta t \, \sigma b_{i,j,k+1} = w_{i,j,k} - \Delta t \, \kappa $$

Cambiando $ \delta_u = \displaystyle \Delta t \, D_u $

$$ (1+ 2\frac{\delta_u}{\Delta x^2} + 2 \frac{\delta_u}{\Delta y^2 } - \frac{\delta_u}{D_u}  \, \lambda + \frac{\delta_u}{D_u} \, w_{i,j,k+1}^2) \, w_{i,j,k+1} - \frac{\delta_u \, w_{i+1, j, k+1}}{\Delta x^2} - \frac{\delta_u \, w_{i-1,j,k+1}}{\Delta x^2} - \frac{\delta_u \, w_{i, j+1, k+1}}{\Delta y^2} - \frac{\delta_u \, w_{i,j-1,k+1}}{\Delta y^2} + \frac{\delta_u}{D_u} \, \sigma b_{i,j,k+1} = w_{i,j,k} - \frac{\delta_u}{D_u} \, \kappa $$

Agrupando incógnitas en la segunda ecuación

$$ b_{i,j,k+1} - \frac{\Delta t \, D_v \, b_{i+1,j,k+1}}{\Delta x^2 \, \tau}  + 2 \frac{\Delta t \, D_v \, b_{i,j,k+1}}{\Delta x^2 \, \tau}  - \frac{\Delta t \, D_v \, b_{i-1,j,k+1}}{\Delta x^2 \, \tau}  - \frac{\Delta t \, D_v \, b_{i,j+1,k+1}}{\Delta y^2 \, \tau}  + 2 \frac{\Delta t \, D_v \, b_{i,j,k+1}}{\Delta y^2 \, \tau} - \frac{\Delta t \, D_v \, b_{i,j-1,k+1}}{\Delta y^2 \, \tau} - \frac{\Delta t}{\tau} \, w_{i,j,k+1} + \frac{\Delta t \, b_{i,j,k+1}}{\tau} = b_{i,j,k} $$

$$ (1+ 2 \frac{\Delta t \, D_v}{\Delta x^2 \, \tau} + 2 \frac{\Delta t \, D_v}{\Delta y^2 \, \tau} + \frac{\Delta t}{\tau}) \, b_{i,j,k+1} - \frac{\Delta t \, D_v}{\Delta x^2 \, \tau} \, b_{i+1,j,k+1}   - \frac{\Delta t \, D_v}{\Delta x^2 \, \tau} \, b_{i-1,j,k+1} - \frac{\Delta t \, D_v}{\Delta y^2 \, \tau} \, b_{i,j+1,k+1}  - \frac{\Delta t \, D_v}{\Delta y^2 \, \tau} \, b_{i,j-1,k+1} - \frac{\Delta t}{\tau} \, w_{i,j,k+1} = b_{i,j,k} $$

Cambiando $ \delta_v = \displaystyle \frac{\Delta t \, D_v}{\tau} $

$$ (1+ 2 \frac{\delta_v}{\Delta x^2} + 2 \frac{\delta_v}{\Delta y^2} + \frac{\delta_v}{D_v}) \, b_{i,j,k+1} - \frac{\delta_v}{\Delta x^2} \, b_{i+1,j,k+1}   - \frac{\delta_v}{\Delta x^2} \, b_{i-1,j,k+1} - \frac{\delta_v}{\Delta y^2} \, b_{i,j+1,k+1}  - \frac{\delta_v}{\Delta y^2} \, b_{i,j-1,k+1} - \frac{\delta_v}{D_v} \, w_{i,j,k+1} = b_{i,j,k} $$

El error de aproximación es de: $O(\Delta t) + O(\Delta x^2) + O(\Delta y^2)$

En resumen, usando $ \delta_u = \displaystyle \Delta t \, D_u $ y $ \delta_v = \displaystyle \frac{\Delta t \, D_v}{\tau} $ nos quedan finalmente el sistema de ecuaciones no-lineales

$$ (1+ 2\frac{\delta_u}{\Delta x^2} + 2 \frac{\delta_u}{\Delta y^2 } - \frac{\delta_u}{D_u}  \, \lambda + \frac{\delta_u}{D_u} \, w_{i,j,k+1}^2) \, w_{i,j,k+1} - \frac{\delta_u \, w_{i+1, j, k+1}}{\Delta x^2} - \frac{\delta_u \, w_{i-1,j,k+1}}{\Delta x^2} - \frac{\delta_u \, w_{i, j+1, k+1}}{\Delta y^2} - \frac{\delta_u \, w_{i,j-1,k+1}}{\Delta y^2} + \frac{\delta_u}{D_u} \, \sigma b_{i,j,k+1} = w_{i,j,k} - \frac{\delta_u}{D_u} \, \kappa $$

$$ (1+ 2 \frac{\delta_v}{\Delta x^2} + 2 \frac{\delta_v}{\Delta y^2} + \frac{\delta_v}{D_v}) \, b_{i,j,k+1} - \frac{\delta_v}{\Delta x^2} \, b_{i+1,j,k+1}   - \frac{\delta_v}{\Delta x^2} \, b_{i-1,j,k+1} - \frac{\delta_v}{\Delta y^2} \, b_{i,j+1,k+1}  - \frac{\delta_v}{\Delta y^2} \, b_{i,j-1,k+1} - \frac{\delta_v}{D_v} \, w_{i,j,k+1} = b_{i,j,k} $$

### b) Implementación de esquema implícito

In [ ]:
def solve_backward(u0, v0, dx, dy, dt, Tf, Du, Dv, lamb, tau, sigma, kappa):
    Nx = 10;
    Ny = 10;
    Nt = Tf/dt;
    w = np.zeros([Nt, Nx, Ny])
    v = np.zeros([Nt, Nx, Ny])
    u[0,:,:] = u0;
    v[0,:,:] = v0;
def func(w,b):
    # b[0], w[0] => i  , j-1
    # b[1], w[1] => i-1, j
    # b[2], w[2] => i  , j
    # b[3], w[3] => i+1, j
    # b[4], w[4] => i  , j+1
    return np.array([(1+2*delta_u / delta_x**2 +2 * delta_u/delta_y**2-delta_u*lamb/Du + (delta_u*w[2]**2)/Du)*w[2]
                    -delta_u*w[3]/delta_x**2-delta_u*w[1]/delta_x**2-delta_u*w[4]/delta_y**2-delta_u*w[0]/delta_y**2
                    +delta_u*sigma*b[2]/Du + delta_u*kappa/Du - w_ijk
                     ,
                     (1+2*delta_v/delta_x**2+2*delta_v/delta_y**2+delta_v/Dv)*b[2]-delta_v*b[3]/delta_x**2
                     -delta_v*b[1]/delta_x**2-delta_v*b[4]/delta_y**2-delta_v*b[0]/delta_y**2-delta_v*w[2]/Dv
                     -b_ijk
                    ]);
sol = fsolve(func)

### c) Profiling

In [ ]:
t_f = 1000
h   = 0.02
t   = np.linspace(0, t_f, (t_f/h)+1)
#timeit
for name in methods:
    print(name + ":")
    %timeit methods[name](lorenz, Y_0, t, h)

In [ ]:
for name in methods:
    print(name + ":")
    %memit methods[name](lorenz, Y_0, t, h)

Del analisis de estos resultados podemos notar que la mejora en la simulación se paga con
con tiempo computacional pues el método más lento es Runge Kutta de cuarto orden seguido
por punto medio y por último el método de Euler que solo demora unos milisegundos.
Aún así, combiene usar `RK4` o `Mid Point` pues sus resultados son mucho mejores como notamos 
en la sección anterior.

Con respecto al uso de memoria no hay mucho que decir pues los tres métodos presentan un 
incremento de 0.00 MiB, lo que significa que gastan muy poca memoria como para detectar el 
cambio. 

<div id="se1d">
### d) Análisis de estabilidad
Los códigos utilzados en esta sección pueden ser vistos en el [anexo](#an5.3). Primero calculamos la data a utilizar:

In [ ]:
Y_0 = np.array([1,1,1])
t_f = 50
hs = [0.1, 0.05, 0.03, 0.02, 0.01]
ts = np.array([np.linspace(0, t_f, (t_f/h)+1) for h in hs])

stability_data = {"Euler": {},
                  "MidPoint": {},
                  "RK4": {} }

for i, h in enumerate(hs):
    for name in methods:
        stability_data[name][h] = methods[name](lorenz, Y_0, ts[i], h)

Obtenemos los valores propios multiplicados por h:

In [ ]:
h_eig_valsR = {"Euler": {}, "MidPoint": {}, "RK4": {} }
h_eig_valsI = {"Euler": {}, "MidPoint": {}, "RK4": {} }

for i, h in enumerate(hs):
    for name in methods:
        h_eig_valsR[name][h] = np.float64(np.ones( stability_data[name][h].shape[0]*stability_data[name][h].shape[1] ))
        h_eig_valsI[name][h] = np.float64(np.zeros( stability_data[name][h].shape[0]*stability_data[name][h].shape[1] ))
        for j, v in enumerate(stability_data[name][h]):
            if not(True in np.isnan(v) or True in np.isinf(v)): #Si el valor es inf o nan pasará a ser 1.
                arr = h * get_eigvals_lorenz(v)  # Multiplicamos por h
                for k in range(3):
                    h_eig_valsR[name][h][3*j+k] = np.real(arr[k])  #Parte real.
                    h_eig_valsR[name][h][3*j+k] = np.imag(arr[k])  #Parte imaginaria.

In [ ]:
colors = ['.r', '.g', '.b']
for h in hs:
    plt_stability_region(colors, h_eig_valsR, h_eig_valsI, h)

Tengamos en cuenta que por la implementación, cuando la matriz tiene un valor `Non a Number` (`NaN`) o infinito,
se graficará un punto en $1 + 0i$, por lo que ese será otro indicador de error.

Por los gráficos notamos como claramente mientras el valor de $h$ disminuye los puntos tienden a
quedar dentro de las regiones de estabilidad.

Para `RK4` vemos como desde $h<0.1$ el método debería dar resultados correctos (Puntos rojos dentro del area roja).

Para `MidPoint` la gráfica queda dentro de la región de estabilidad desde $h<0.05$
(Puntos verdes dentro del area verde).

El peor caso sin duda es para el método de Euler pues no entra en la región de estabilidad hasta 
que $h < 0.02$ (Puntos azules dentro del area azúl).

### e) Implementación paralela
Ver [anexo 2](#an2)

### f) Simulación de dos trayetorias

In [ ]:
d  = 0.00001
tf = 50
h  = 0.02
t   =  np.linspace(0, tf, (tf/h)+1)

#Condicion inicial a eleccion:
Y0 = np.array([[1, 1, 1],
               [1+d, 1+d, 1+d]])

parallel_dataRK4   = parallel_RK4_int(lorenz, Y0, t, h)
parallel_dataMP    = parallel_mp_int(lorenz, Y0, t, h)
parallel_dataEuler = parallel_euler_int(lorenz, Y0, t, h)

In [ ]:
animate_lorenz(parallel_dataEuler, 2)
animate_lorenz(parallel_dataMP, 2)
animate_lorenz(parallel_dataRK4, 2)

En este caso, a pesar de que las condiciones iniciales son muy cercanas, las gráficas se comportan
de manera diferente despues de un tiempo. Esto demuestra que el atractor de Lorenz con los parametros 
seleccionados es caótico, soluciones diferentes se obtienen por una variación pequeña.

Cada uno de los metodos presenta este comportamiento, a pesar que el metodo de euler es menos preciso
las graficas comienzan a diverger más o menos en el mismo momento que en los otros dos.

### g) Simulación de veinte trayetorias con RK4

In [ ]:
#20 trayectorias entre 0.99999 y 1.00001:
Y0 = np.array([[ (1-d)+(d*i/10)  for _ in range(3)] for i in range(20)])

big_parallel_dataRK4 = parallel_RK4_int(lorenz, Y0, t, h)

In [ ]:
animate_lorenz(big_parallel_dataRK4, 20)

Igual que en el caso anterior podemos ver como el comportamiento del atractor es diferente para cada
valor a pesar de que están muy cercanos entre ellos. 

Despues de cierto tiempo todas las trayectorias comienzan a moverse de manera diferente. Aunque en 
un principio todas seguían el mismo camino, cuando se separan comienzan a tener una pequeña distancia
entre ellas que mientras pasa el tiempo se incrementa mostrando las diferencias en su comportamiento.

<div id='se2'/>
## 2.- BVP de orden superior

### a) Transformación a IVP acoplado
$$ y^{(4)}(x) = -y(x) $$
$$ BC: \{ y(0)= 0, y(2) =1, y'(2)=-1, y''(2)=1 \} $$

Se hace un cambio de variable:
$$
\begin{matrix}
y_1 & = & y \\
y_2 & = & y' \\
y_3 & = & y''\\
y_4 & = & y^{(3)}
\end{matrix} \quad \Longrightarrow \quad
\begin{matrix}
y_1' & = & y' \\
y_2' & = & y'' \\
y_3' & = & y^{(3)}\\
y_4' & = & y^{(4)}
\end{matrix} \quad \Longrightarrow \quad
\begin{matrix}
y_1' & = & y_2 \\
y_2' & = & y_3 \\
y_3' & = & y_4 \\
y_4' & = & -y_1
\end{matrix}
$$

Luego: $ y_1(0)=0, y_2(0) = \alpha , y_3(0) = \beta, y_4(0) = \gamma $

Donde $\alpha$, $\beta$ y $\gamma$ son valores tales que permiten que $y_1(2) = 1$, $y_2(2)=-1$ y $y_3(2)=1$ con algún algoritmo.

### b) Propuesta de cuarta derivada y discretización
Para discretizar la ecuación anterior se debe recurrir a Backward Difference (BD), Forward Difference (FD) y Central Difference (CD). Para aproximar la cuarta derivada se van a necesitar 5 puntos como mínimo para la discretización. 

Lo primero es aproximar las primeras derivadas en los 5 puntos con BD y FD. Luego se aproximarán las segundas derivadas en los puntos intermedios de los 5 puntos (los 3 puntos intermedios). 

Para calcular las segundas derivadas se hará un CD entre la primera derivada del punto anterior calculada con BD y la primera derivada del punto siguiente calculada con FD, así logramos tener las segundas derivadas para los 3 puntos centrales. Tomando como punto central $x$ y intervalos $h$ las derivadas quedan como sigue:

$$ 
\begin{align}
f''(x-h) &= \displaystyle \frac{f(x-2h) - 2f(x-h) + f(x)}{h^2} \\
f''(x)   &= \displaystyle \frac{f(x-h) - 2f(x) + f(x+h)}{h^2} \\
f''(x+h) &= \displaystyle \frac{f(x) - 2f(x+h) + f(x+2h)}{h^2}
\end{align}
$$

Una vez aproximadas las segundas derivadas, aproximaremos las terceras derivadas. Para ello se utilizará un FD y BW.

$$ f^{(3)}(x-h) = \displaystyle \frac{f''(x) - f''(x-h)}{h} = \displaystyle \frac{f(x+h)-3f(x)+3f(x-h)-f(x-2h)}{h^3} $$
$$ f^{(3)}(x+h) = \displaystyle \frac{f''(x+h) - f''(x)}{h} = \displaystyle \frac{f(x+2h)-3f(x+h)+3f(x)-f(x-h)}{h^3} $$

Finalmente, aproximamos la cuarta derivada en $x$ con CD sobre estas dos derivadas.

$$ f^{(4)}(x) = \displaystyle \frac{f^{(3)}(x+h) - f^{(3)}(x-h)}{2h} = \displaystyle \frac{f(x+2h)-4f(x+h)+6f(x)-4f(x-h)+f(x-2h)}{2h^4} $$

Además, debemos aproximar la segunda derivada en el punto final. Para ello tendremos problemas si usamos CD ya que no tendremos los puntos que vienen más a la derecha, para ellos tendremos que usar 2 veces BD para sacar 2 primeras derivadas y aplicar BD sobre ellas, considerando $N$ intervalos obtendremos:

$$ f''(x_{N}) = \displaystyle \frac{y_N-2y_{N-1}+y_{N-2}}{h^2} $$

Ahora, discretizando nuestro problema con la aproximación conseguida y tomando $N+1$ puntos $(x_0,y_0),(x_1,y_1),\cdots ,(x_{N-1},y_{N-1}),(x_{N},y_{N})$ con $h$ como intervalo obtenemos las siguientes ecuaciones:

\begin{align}
y_0   &= 0 \\
y_{i} &= \displaystyle -\frac{6y_{i}-4y_{i+1}+y_{i+2}-4y_{i-1}+y_{i-2}}{2h^4} \\
y_{N} &= 1
\end{align}
Además:
$$ \displaystyle \frac{y_{N}-y_{N-1}}{h} = -1 $$
$$ \displaystyle \frac{y_{N}-2y_{N-1}+y_{N-2}}{h^2} = 1 $$

Reemplazando lo conocido y utilizando $ N + 1$ puntos y $N$ intervalos:

$$ y_i = \displaystyle -\frac{6y_i-4y_{i+1}+1-4y_{i-1}}{2h^4} $$
$$ \displaystyle \frac{1-y_{N-1}}{h} = -1 $$
$$ \displaystyle \frac{1-2y_{N-1}+y_{N-2}}{h^2} = 1 $$

Finalmente tenemos 3 ecuaciones y 3 incógnitas resolvibles con algún algoritmo de resolución de sistemas de ecuaciones lineales.

### c) Implementación de diferencias finitas
Ver [Anexo 4](#an4)

Lo primero que se hace para calcular la discretización por medio de diferencias finitas es obtener los valores de $y$ que son fácilmente calculables $y_0, y_N, y_{N-1}, y_{N-2}$ estos valores se obtienen con 4 de las  5 ecuaciones indicadas. En las demás ecuaciones se reemplazan estos valores obtenidos para que dejen de ser incógnitas y pasen a ser valores conocidos y se forma la matriz con las incógnitas restantes. 

Considerando todas las incógnitas y las condiciones $ y(0)=\alpha_1, y'(2)=\alpha_2, y''(2)=\alpha_3, y(2)=\alpha_4 $ la matriz queda así:

$$
\begin{bmatrix} 
    1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \cdots & 0 \\
    -1 & 4 & -2h^4-6 & 4 & -1 & 0 & 0 & 0 &  & 0 \\
    0 & -1 & 4 & -2h^4-6 & 4 & -1 & 0 & 0 &  & 0 \\
    0 & 0 & -1 & 4 & -2h^4-6 & 4 & -1 & 0 &  & 0 \\
    \vdots & & & & & \vdots & & & & \vdots \\
    0 & \cdots & 0 & 0 & 0 &  -1 & 4 & -2h^4-6 & 4 & -1 \\
    0 & \cdots & 0 & 0 & 0 & 0 & 0 & 0 & -1 & 1 \\
    0 & \cdots & 0 & 0 & 0 & 0 & 0 & 1 & -2 & 1 \\
    0 & \cdots & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1
\end{bmatrix}
\left[ 
\begin{array}{c} 
y_0 \\ 
y_1 \\
y_2 \\
y_3 \\
\vdots \\
y_{N-3} \\
y_{N-2} \\
y_{N-1} \\
y_N \\
\end{array} \right] 
=
\left[ \begin{array}{c}
\alpha_1 \\ 
0 \\
0 \\
0 \\
\vdots \\
0 \\
\alpha_2 \, h \\
\alpha_3 \, h^2 \\
\alpha_4 \\
\end{array} \right]
$$

Luego se calculan los valores de 4 incógnitas como:
\begin{align}
y_0     &= \alpha_1 \\
y_{N-2} &= \alpha_3 \, h^4 + \alpha_4 -2\alpha_2 \, h \\
y_{N-1} &= \alpha_4 - \alpha_2 \, h \\
y_N     &= \alpha_4
\end{align}

La matriz eliminando las 4 incógnitas fácilmente calculables queda así:
$$
\begin{bmatrix} 
    4 & -2h^4-6 & 4 & -1 & 0 & 0 & 0 & 0 & \cdots & 0 \\
    -1 & 4 & -2h^4-6 & 4 & -1 & 0 & 0 & 0 &  & 0 \\
    0 & -1 & 4 & -2h^4-6 & 4 & -1 & 0 & 0 &  & 0 \\
    0 & 0 & -1 & 4 & -2h^4-6 & 4 & -1 & 0 &  & 0 \\
    \vdots & & & & & \vdots & & & & \vdots \\
    0 & \cdots & 0 & 0 & 0 &  -1 & 4 & -2h^4-6 & 4 & -1 \\
    0 & \cdots & 0 & 0 & 0 & 0 & -1 & 4 & -2h^4-6 & 4 \\
    0 & \cdots & 0 & 0 & 0 & 0 & 0 & -1 & 4 & -2h^4-6 \\
    0 & \cdots & 0 & 0 & 0 & 0 & 0 & 0 & -1 & 4
\end{bmatrix}
\left[ 
\begin{array}{c} 
y_1 \\
y_2 \\
y_3 \\
y_4 \\
\vdots \\
y_{N-6} \\
y_{N-5} \\
y_{N-4} \\
y_{N-3} \\
\end{array} \right] 
=
\left[ \begin{array}{c}
\alpha_1 \\ 
0 \\
0 \\
0 \\
\vdots \\
0 \\
y_{N-2}\\
y_{N-1}-4y_{N-2}\\
y_N-4y_{N-1}+(2h^4+6)y_{N-2}\\
\end{array} \right]
$$

Al observar la matriz se puede notar que es casi una matriz triangular superior por lo que es fácilmente convertible en una. Entonces, con el fin de aprovechar las ventajas de una matriz triangular superior convertimos la matriz $A$ en una de ellas por operaciones filas, el vector $b$ obviamente también se ve afectado.

Una vez tenemos nuestro sistema de ecuaciones lineales con una matriz triangular superior lo resolvemos con Backward Substitution obteniendo la solución final.

<div id="se2d">
### d) Ejecución del algoritmo

In [ ]:
#Límite izquierdo de x
x0 = 0
#Límite derecho de x
xN = 2
#Valor de y en el límite izquierdo
y0 = 0
#Valor de y en el límite derecho
yN = 1
#Valor de la derivada en el límite derecho
ydN = -1
#Valor de la segunda derivada en el límite derecho
yddN = 1

for N in [4, 8, 15, 100, 10000]:
    f = fin_dif(N, x0, xN, y0, yN, ydN, yddN)
    plot_2_D(f)

Se puede apreciar claramente como al aumentar la cantidad de intervalos $N$ la precisión de la función a aproximar aumenta.

En la primera figura ($N=4$) se puede ver una gráfica muy discretizada donde las líneas no se ven suaves.

En la segunda figura ($N=8$) ya se puede observar mejor la forma de la función, quizás esta discretización ya baste para algunas aplicaciones obteniendo un valor de $y$ bastante cercano al real a pesar de que en la gráfica aún se pueden observar los límites de cada intervalo.

En la tercera figura ($N=15$) con mucho detalle se pueden observar aún los intervalos de discretización pero ya tenemos una función que se puede apreciar correctamente.

En la cuarta ($N=100$) y quinta ($N=10000$) figura ya no se pueden apreciar a simple vista donde comienza y termina un intervalo de discretización, la precisión de las aproximación es bastante alta en estos gráficos.

## Conclusiones

En este informe se resolvieron ecuaciones diferenciales por métodos computacionales y se pudo determinar 
las carácteristicas que influyen en el desempeño de cada uno de los métodos utilizados y las condiciones
que deben cumplir los problemas para obtener buenos resultados.

Particularmente en el atractor de Lorenz notamos que para determinadas condiciones iniciales es muy difícil
predecir el comportamiento de un fenómeno (caótico) y por ello las concluciones que podemos sacar sin una 
simulación son poco acertadas. Despues de graficar logramos una mejor comprensión del fenomenó en sí y 
concluimos que, a pesar de que el error del método utilizado para hacer la estimación debe ser tomado en
cuenta, el comportamiento de las ecuaciones no cambia.

Por otro lado, se ha aproximado una derivada de cuarto orden con una combinación de Backward, Central y Forward Difference y luego se ha aproximádo una solución numérica a una ecuación que incluía una cuarta derivada.
 Se generó un sistema de ecuaciones lineales y se expresó matricialmente. Aprovechando las cualidades de este sistema se resolvió con Backward Substitution. Finalmente, se analizó y graficó los resultados para distintos valores de precisión.

## Referencias
<div id='ref1'\> [1] [IPython Notebook Documentation](http://ipython.readthedocs.org/en/stable/) IPython Documentation. *desc*. Revisado 16/12/2015
<div id='ref1'\> [2] [Python Documentation](https://docs.python.org/3/) Python Documentation. *desc*. Revisado 16/12/2015

## Anexos
Comenzamos cargando los módulos necesarios:

In [ ]:
import numpy as np
from scipy import integrate
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import cnames
from matplotlib import animation

%matplotlib notebook
#%load_ext memory_profiler

<div id='an1'\> 
### Métodos implementados:

In [ ]:
def euler_int(func, y0, t, h):
    ys = np.zeros([t.shape[0],y0.shape[0]])
    ys = np.float64(ys)
    ys[0,:] = y0
    for i in range(t.shape[0]-1):
        ys[i+1,:] = ys[i,:] + h*func(t[i], ys[i,:])
    return ys

In [ ]:
def mp_int(func, y0, t, h):
    h2 = h/2
    ys = np.zeros([t.shape[0],y0.shape[0]])
    ys = np.float64(ys)
    ys[0,:] = y0
    for i in range(t.shape[0]-1):
        k1 = ys[i,:] + h2*func(t[i], ys[i,:])
        ys[i+1,:] = ys[i,:] + h*func(t[i] + h2 , k1)
    return ys

In [ ]:
def RK4_int(func, y0, t, h):
    h2 = h/2
    ys = np.zeros([t.shape[0],y0.shape[0]])
    ys = np.float64(ys)
    ys[0,:] = y0
    for i in range(t.shape[0]-1):
        k1 = func(t[i], ys[i,:])
        k2 = func(t[i] + h2, ys[i,:] + h2*k1)
        k3 = func(t[i] + h2, ys[i,:] + h2*k2)
        k4 = func(t[i] + h,  ys[i,:] + h*k3)
        ys[i+1,:] = ys[i,:] + (h/6)*(k1 + 2*(k2 + k3) + k4)
    return ys

In [ ]:
methods = {"Euler": euler_int, "MidPoint": mp_int, "RK4": RK4_int}

<div id='an2'\> 
### Métodos paralelos:

In [ ]:
#Retorna una función que ejecuta la función original en una matriz
def mk_parallel_func(func):
    def new_func(t, f, r): #r es un array ya creado, con las dimensiones correctas.
        for i in range(r.shape[1]):
            r[:,i] = func(t, f[:,i])
        return r
    return new_func

In [ ]:
def parallel_euler_int(func, y0, t, h):
    ys = np.float64(np.zeros([t.shape[0], y0.shape[1], y0.shape[0]]))
    rf = np.float64(np.zeros([y0.shape[1], y0.shape[0]]))
    ys[0,:,:] = y0.T
    vf = mk_parallel_func(func)
    for i in range(t.shape[0]-1):
        ys[i+1,:,:] = ys[i,:,:] + h * vf(t[0], ys[i,:,:], rf)#rf
    return ys

In [ ]:
def parallel_mp_int(func, y0, t, h):
    h2 = h/2
    ys = np.float64(np.zeros([t.shape[0], y0.shape[1], y0.shape[0]]))
    rf = np.float64(np.zeros([y0.shape[1], y0.shape[0]]))
    ys[0,:,:] = y0.T
    vf = mk_parallel_func(func)
    for i in range(t.shape[0]-1):
        k1 = ys[i,:,:] + h2*vf(t[i], ys[i,:,:], rf)
        ys[i+1,:,:] = ys[i,:,:] + h*vf(t[i] + h2 , k1, rf)
    return ys

In [ ]:
def parallel_RK4_int(func, y0, t, h):
    h2 = h/2
    ys = np.float64(np.zeros([t.shape[0], y0.shape[1], y0.shape[0]]))
    rf = np.float64(np.zeros([y0.shape[1], y0.shape[0]]))
    ys[0,:,:] = y0.T
    vf = mk_parallel_func(func)
    for i in range(t.shape[0]-1):
        k1 = np.copy( vf(t[i], ys[i,:,:], rf) )
        k2 = np.copy( vf(t[i] + h2, ys[i,:,:] + h2*k1, rf) )
        k3 = np.copy( vf(t[i] + h2, ys[i,:,:] + h2*k2, rf) )
        k4 = np.copy( vf(t[i] + h,  ys[i,:,:] + h*k3,  rf) )
        ys[i+1,:,:] = ys[i,:,:] + (h/6)*(k1 + 2*(k2 + k3) + k4)
    return ys

<div id='an3'\> 
### Visualización de resultados

In [ ]:
"""
-> x_t
   arreglo con las posiciones (x,y,z) de las trayectorias. Este puede ser
   de dos dimensiones (len(t), 3) para una sola trayectoria, o puede ser 
   de tres dimensiones (len(t), 3, N_trajectories) para mas de una trayectoria
-> N_trajectories
   numero de trayectorias a visualizar
-> xlim,ylim,zlim
   limites en los ejes x,y,z de la animacion
-> rotate
   rotar mientras anima para mejores perspectivas
"""
def animate_lorenz(x_t, N_trajectories, xlim=(-20,70), ylim=(-35,35), zlim=(-35,35), rotate=False):
    #setting it to correct format
    if x_t.ndim==2:
        x_t = np.array([x_t])
    elif x_t.ndim==3: 
        x_t = np.rot90(x_t).T
    else:
        return -1
    #setting the number of frames
    frames = max(x_t.shape)
    frames /= 2
    frames = int(frames)
    #set up figure & 3D axis for animation
    fig = plt.figure()
    ax = fig.add_axes([0, 0, 1, 1], projection='3d')
    #ax.axis('off')

    #choose a different color for each trajectory
    colors = plt.cm.jet(np.linspace(0, 1, N_trajectories))

    #set up lines and points
    lines = sum([ax.plot([], [], [], '-', c=c)
                 for c in colors], [])
    pts = sum([ax.plot([], [], [], 'o', c=c)
               for c in colors], [])

    #prepare the axes limits
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    ax.set_zlim(zlim)

    #set point-of-view: specified by (altitude degrees, azimuth degrees)
    ax.view_init(30, 0)

    #initialization function: plot the background of each frame
    def init():
        for line, pt in zip(lines, pts):
            line.set_data([], [])
            line.set_3d_properties([])

            pt.set_data([], [])
            pt.set_3d_properties([])
        return lines + pts

    #animation function.  This will be called sequentially with the frame number
    def animate(i):
        # we'll step two time-steps per frame.  This leads to nice results.
        i = (2 * i) % x_t.shape[1]

        for line, pt, xi in zip(lines, pts, x_t):
            x, y, z = xi[:i].T
            line.set_data(x, y)
            line.set_3d_properties(z)

            pt.set_data(x[-1:], y[-1:])
            pt.set_3d_properties(z[-1:])

        if rotate: ax.view_init(30, 0.3 * i)
        fig.canvas.draw()
        return lines + pts

    #instantiate the animator.
    anim = animation.FuncAnimation(fig, animate, init_func=init,
                                   frames=frames, interval=30, blit=True, repeat=False)
    #show it
    ax.set_title('Lorenz Simulation')
    ax.set_xlabel('x(t)')
    ax.set_ylabel('y(t)')
    ax.set_zlabel('z(t)')
    plt.show()

<div id='an4'\> 
### Implementación de diferencias finitas

In [ ]:
def fin_dif(N, x0, xN, y0, yN, ydN, yddN):
    if (N<4):
        print('Error: Se necesita al menos 4 intervalos (5 puntos).');
        return;
    #inicializando vector de y con ceros.
    y = np.zeros([N+1,1])
    #tamaño del intervalo
    h = (xN-x0)/N;
    x = np.linspace(x0, xN, N+1);
    
    #estableciendo valores conocidos
    y[0] = y0;
    y[N] = yN;
    y[N-1] = y[N]-ydN*h
    y[N-2] = yddN*h**4+yN-2*ydN*h;
    
    M = (N+1)-4;
    A = np.zeros([M, M]);
    b = np.zeros([M, 1]);
    
    #creando matriz A
    for i in range(M):
        if (i >= 1):
            A[i,i-1] = -1;
        A[i,i] = 4;
        if (i <= M-2):
            A[i,i+1] = -2*h**4-6;
        if (i <= M-3):
            A[i,i+2] = 4;
        if (i <= M-4):
            A[i,i+3] = -1;
            
    #creando vector b
    b[0] = y[0];
    if (M >= 3):
        b[M-3] = b[M-3] + y[N-2];
    if (M >= 2):
        b[M-2] = b[M-2] + y[N-1] - 4*y[N-2];
    b[M-1] = b[M-1] + y[N] - 4*y[N-1] - (-2*h**4-6)*y[N-2];
    
    #convirtiendo la matriz A en triangular superior
    for i in range(M-1):
        A[i+1, :] = A[i+1, :]+A[i, :]/A[i,i];
        b[i+1] = b[i+1]+b[i]/A[i,i];
    
    # Resolviendo el sistema de ecuaciones lineales con backward substitution
    for i in range(M-1,-1,-1):
        j = i+1;
        if (i== M-1):
            y[j] = b[i]/A[i,i]
        elif (i== M-2):
            y[j] = (b[i]-A[i,i+1]*y[j+1])/A[i,i];
        elif (i== M-3):
            y[j] = (b[i]-A[i,i+1]*y[j+1]-A[i,i+2]*y[j+2])/A[i,i];
        else:
            y[j] = (b[i]-A[i,i+1]*y[j+1]-A[i,i+2]*y[j+2]-A[i,i+3]*y[j+3])/A[i,i];
            
    #formateando output
    f = np.zeros([N+1, 2]);
    f[:,0] = x;
    f[:,1] = y[:,0];
    return f;

<div id='an5'\> 
### Utilidades
Método que permite dibujar gráficos para la [respuesta 2.d](#se2d)

In [ ]:
def plot_2_D(f):
    #Tamaño de los graficos
    FIGSIZE = (12, 6)
    plt.figure(figsize=FIGSIZE)
    plt.plot(f[:,0], f[:,1])
    plt.title('Gráfico de y(x) ')
    plt.xlabel("x")
    plt.ylabel("y")
    plt.show()

Calculo de valores propios del [Atractor de Lorenz](#se1.1):

In [ ]:
def get_eigvals_lorenz(Y_0):
    A = np.array([
            [-10, 10, 0],
            [28-Y_0[2], -1, -Y_0[0]],
            [Y_0[1], Y_0[0], -8/3]
        ])
    return np.linalg.eigvals(A)

<div id='an5.3'\> 
Código extraido de `U3W2_AdvancedApplications.ipynb` para el análisis de las regiones de estabilidad.
Utilizado en la [respuesta 1.d](#se1d)

In [ ]:
def zplot2(z, ax=plt.gca(), lw=1.5, line_color='k'):
  ax.plot(np.real(z), np.imag(z), line_color, lw=lw)
def runge_kutta_stability_regions():
  z = np.exp(1j * np.pi * np.arange(201)/100.)
  r = z-1
  d = 1-1./z;
  # Order 1
  W1, W2, W4 = [0], [0], [0]
  for zi in z[1:]:
    W1.append( W1[-1]-(1.+W1[-1]-zi) )
  for zi in z[1:]:
    W2.append( W2[-1]-(1+W2[-1]+.5*W2[-1]**2-zi**2)/(1+W2[-1]) )
  for zi in z[1:]:
    num = (1+W4[-1]+.5*W4[-1]**2+W4[-1]**3/6+W4[-1]**4/24-zi**4)
    den = (1+W4[-1]+W4[-1]**2/2+W4[-1]**3/6.)
    W4.append( W4[-1] - num/den )
  return W1, W2, W4
W1,W2,W4=runge_kutta_stability_regions()

In [ ]:
def plt_stability_region(colors, real, imag, h):
    fig = plt.figure()
    ax=fig.gca()
    zplot2(W1,ax,line_color='r')
    zplot2(W2,ax,line_color='g')
    zplot2(W4,ax,line_color='b')
    #Cambio de orden para una mejor visualizacion
    for i, name in enumerate(["Euler", "MidPoint", "RK4"]):
        plt.plot(real[name][h], imag[name][h], colors[i], ms=10, alpha=.4, label=name)
    ax.axis("equal")
    ax.axis([-5, 2, -3.5, 3.5])
    ax.grid("on")
    ax.set_title("Region de estabilidad para h="+str(h))
    plt.legend(loc=3, borderaxespad=0.)
    plt.show()